In [2]:
from music21 import *

In [ ]:
def harmonize(melody):
    '''
    Given a melody as a music21 Score object, returns a new Score object involving
    a possible harmonization of the melody.
    '''
    # > let's at first assume we're assigning a chord every beat for simplicity's sake
    
    melody_key = melody.analyze('key')
    
    part1 = analyze_romans(melody, melody_key)
    
    # chord[last melody note] = tonic
    
    # working backwards from last chord, iterate through each beat and call find_roman_numeral()
    part2 = fill_in_chords(part1)
    
    return part2